In [14]:
import math
import time

import numpy as np
from pyproj import Geod
from geographiclib.geodesic import Geodesic

from notebooks.ls_association import SOUND_MODEL


In [15]:
#plus rapide
geod = Geod(ellps="WGS84")  # use WGS84 ellipsoid

lat1, lon1, lat2, lon2 = 3.5, 63.5,-25.0, 70.0
t0 = time.time()
for i in range(50000):
    # Compute forward and back azimuths, and total distance
    geod = Geod(ellps="WGS84")  # use WGS84 ellipsoid
    az12, az21, total_distance = geod.inv(lon1, lat1, lon2, lat2, return_back_azimuth=False)
t = time.time() -t0
print(t)
print(az12, az21, total_distance )

0.12783241271972656
167.7795687813548 166.5272109081273 3230437.886351984


In [16]:
#plus lent
t0 = time.time()
for i in range(50000):
# Ellipsoïde WGS84
    geod = Geodesic.WGS84
    lat1, lon1, lat2, lon2 = 3.5, 63.5,-25.0, 70.0
    # Calcul avec dérivées partielles
    result = geod.Inverse(lat1, lon1, lat2, lon2)
t = time.time() -t0
print(t)
print(result)

1.7769298553466797
{'lat1': 3.5, 'lon1': 63.5, 'lat2': -25.0, 'lon2': 70.0, 'a12': 29.111892464274877, 's12': 3230437.886351984, 'azi1': 167.7795687813548, 'azi2': 166.5272109081273}


In [17]:
a = geod.a
f = geod.f
b = a-f*a
e = math.sqrt(a**2-b**2)/a

In [18]:
e2 = e**2
ep2 = (e2)/(1-e2)
w=1/math.sqrt(1+ep2*math.cos(math.radians(lat2))**2)
rho=(a*w**3)/(1-f)
mu=(a*w**3)/(1-f)
R=mu*math.cos(lat2)

In [19]:
rho = a*(1-e2)**2/((w**3 * 1-e2*math.sin(math.radians(lat2))**2)**1.5)
print(rho)
N2 = a / math.sqrt(1-e2*math.sin(math.radians(lat2))**2)
print(N2)
M2 = a*(1-e2) / math.pow(1-e2*math.sin(math.radians(lat2))**2,1.5)
print(M2)

6383221.591901079
6381953.457155743
6346818.85872043


In [20]:
geod = Geod(ellps="WGS84")
def theoretical_derivatives(lat1, lon1, lat2, lon2):

    azi1, azi2, total_distance = geod.inv(lon1, lat1, lon2, lat2, return_back_azimuth=False)

    # Conversion degrés → radians
    lat2_rad = np.radians(lat2)
    azi2_rad = np.radians(azi2)

    # Paramètres ellipsoïdaux
    a = geod.a
    f = geod.f
    e2 = f * (2 - f)

    # Composantes trigonométriques
    sin_lat2 = np.sin(lat2_rad)
    cos_lat2 = np.cos(lat2_rad)
    sin_azi2 = np.sin(azi2_rad)
    cos_azi2 = np.cos(azi2_rad)

    # Rayons de courbure au point B
    nu2 = 1 - e2 * sin_lat2**2
    N2 = a / np.sqrt(nu2)                    # Rayon de courbure normal au point 2
    M2 = a * (1 - e2) / (nu2**1.5)           # Rayon de courbure méridien au point 2

    # === DÉRIVÉES DE LA DISTANCE s12 ===
    # Basées sur la géométrie différentielle de l'ellipsoïde
    # Ces formules sont directes et bien établies
    ds12_dlat2 = M2 * cos_azi2 * 180/np.pi    # ∂s₁₂/∂φ₂ (composante méridienne) en m/degres
    ds12_dlon2 = N2 * cos_lat2 * sin_azi2 *180/np.pi   # ∂s₁₂/∂λ₂ (composante parallèle) en m/degres

    return ds12_dlat2,ds12_dlon2
for i in range(50000):
    ds12_dlat2,ds12_dlon2=theoretical_derivatives(lat1, lon1, lat2, lon2)
print(ds12_dlat2,ds12_dlon2)

-353638644.6043217 77210651.2559508


In [21]:
eps = 0.001
azi1, azi2, total_distance0 = geod.inv(lon1, lat1, lon2, lat2-eps, return_back_azimuth=False)
azi1, azi2, total_distance1 = geod.inv(lon1, lat1, lon2, lat2+eps, return_back_azimuth=False)
ds_dlat = (total_distance1 -total_distance0) / (2*eps)
print(ds_dlat)
eps = 0.002
azi1, azi2, total_distance0 = geod.inv(lon1, lat1, lon2-eps, lat2, return_back_azimuth=False)
azi1, azi2, total_distance1 = geod.inv(lon1, lat1, lon2+eps, lat2, return_back_azimuth=False)
ds_dlon = (total_distance1 -total_distance0) / (2*eps)
print(ds_dlon)

-107724.49145349674
23519.709315500222


In [22]:
geod = Geod(ellps="WGS84")
def theoretical_derivatives(lat1, lon1, lat2, lon2):

    azi1, azi2, total_distance = geod.inv(lon1, lat1, lon2, lat2, return_back_azimuth=False)
    # Conversion degrés → radians
    lat2_rad = np.radians(lat2)
    azi2_rad = np.radians(azi2)

    # Paramètres ellipsoïdaux
    a = geod.a
    f = geod.f
    e2 = f * (2 - f)

    # Composantes trigonométriques
    sin_lat2 = np.sin(lat2_rad)
    cos_lat2 = np.cos(lat2_rad)
    sin_azi2 = np.sin(azi2_rad)
    cos_azi2 = np.cos(azi2_rad)

    # Rayons de courbure au point B
    nu2 = 1 - e2 * sin_lat2**2
    N2 = a / np.sqrt(nu2)                    # Rayon de courbure normal au point 2
    M2 = a * (1 - e2) / (nu2**1.5)           # Rayon de courbure méridien au point 2

    # === DÉRIVÉES DE LA DISTANCE s12 ===
    # Basées sur la géométrie différentielle de l'ellipsoïde
    # Ces formules sont directes et bien établies
    ds12_dlat2 = M2 * cos_azi2  # ∂s₁₂/∂φ₂ (composante méridienne)
    ds12_dlon2 = N2 * cos_lat2 * sin_azi2  # ∂s₁₂/∂λ₂ (composante parallèle)

    return ds12_dlat2,ds12_dlon2

# Jacobian of the geodesic distance difference wrt unknown position
def Jacobian(lat_i, lon_i, lat0, lon0, lat_x, lon_x):
    n = len(lat_i)
    J = np.zeros((n, 2))

    # Partial derivatives for si
    dsi_dlat,dsi_dlon = theoretical_derivatives(lat_x, lon_x, lat_i, lon_i)
    # Partial derivatives for s0 (reference)
    ds0_dlat,ds0_dlon=theoretical_derivatives(lat_i, lon_i, lat0, lon0)

    # Chain rule: d(si - s0)/dx
    J[:, 0] = dsi_dlat - ds0_dlat  # derivative w.r.t. lat_x
    J[:, 1] = dsi_dlon - ds0_dlon  # derivative w.r.t. lon_x

    return J

# --------------------------------------------------
# Weighted least squares solver
def calc_ls(J, Y, SigmaY):
    n, p = J.shape
    iSigmaY = np.linalg.inv(SigmaY)
    N = J.T @ iSigmaY @ J
    iN = np.linalg.inv(N)
    Xhat = iN @ J.T @ iSigmaY @ Y
    Vhat = J @ Xhat - Y
    sigma02hat = (Vhat.T @ iSigmaY @ Vhat / (n - p)).item(0)
    SigmaXhat = sigma02hat * iN
    return Xhat, sigma02hat, SigmaXhat

# --------------------------------------------------
# Main TDOA solver
def estimate_position_tdoa(lat_i_deg, lon_i_deg, tdoa_obs, speed_of_sound, initial_guess_deg, sigma_obs, max_iter=50):
    # Convert to radians
    lat_i = (lat_i_deg)
    lon_i = (lon_i_deg)
    lat_x, lon_x = (initial_guess_deg)

    # Reference receiver: index 0
    lat0 = np.full(len(lat_i), lat_i[0])
    lon0 = np.full(len(lon_i), lon_i[0])

    lat_x, lon_x = np.full(len(lat_i), lat_x), np.full(len(lon_i), lon_x)
    # Convert TDOA to observed distance differences
    delta_s_obs = speed_of_sound * (tdoa_obs - tdoa_obs[0])
    print("delta_s_obs", delta_s_obs)
    # Observation covariance matrix (homoscedastic)
    SigmaY = np.eye(len(delta_s_obs)) * sigma_obs**2

    for _ in range(max_iter):
        # Compute predicted distances (in radians)
        _,_,si = geod.inv(lon_x, lat_x, lon_i, lat_i)
        _,_,s0 = geod.inv(lon_x, lat_x, lon0, lat0)

        # Predicted difference in meters
        f_pred = (si - s0)

        # Residuals (observed - predicted)
        Y = (delta_s_obs - f_pred).reshape(-1, 1)

        # Jacobian scale to meters
        J = Jacobian(lat_i, lon_i, lat0, lon0, lat_x, lon_x)
        rad2deg = 180.0 / np.pi
        J_scaled = J * rad2deg

        # Solve linear system
        delta_x, sigma0_sq, Sigma_x = calc_ls(J_scaled, Y, SigmaY)

        # Update position
        lat_x += delta_x[0, 0]
        lon_x += delta_x[1, 0]

        # Convergence check
        if np.linalg.norm(delta_x) < 1e-12:
            break

    return np.degrees([lat_x, lon_x]), sigma0_sq, Sigma_x

# --------------------------------------------------
# Example usage
if __name__ == "__main__":

    #attention ces valuers de TDOA sont du CACA
    # Receiver positions (degrees)
    lat_i_deg = np.array([48.85, 48.86, 48.87, 48.84])
    lon_i_deg = np.array([2.35, 2.34, 2.36, 2.33])

    # Simulated TDOA observations (seconds)
    tdoa_obs = np.array([0., 0.0002, 0.006, -0.002])

    # Speed of sound (m/s)
    speed_of_sound = [1500,1480,1500,1509]

    # Initial guess (near Paris)
    initial_guess_deg = [48.855, 2.33]

    # Standard deviation of timing obs (in meters)
    sigma_obs = 10

    # Estimate source position
    pos_est, sigma0, cov_pos = estimate_position_tdoa(lat_i_deg, lon_i_deg, tdoa_obs,
                                                      speed_of_sound, initial_guess_deg, sigma_obs)

    print("Estimated position (lat, lon):", pos_est)
    print("A posteriori variance factor:", sigma0)
    print("Covariance matrix of estimate:\n", cov_pos)


delta_s_obs [ 0.     0.296  9.    -3.018]
Estimated position (lat, lon): [[2799.18858356 2799.18858356 2799.18858356 2799.18858356]
 [ 133.48760879  133.48760879  133.48760879  133.48760879]]
A posteriori variance factor: 5442.031021092539
Covariance matrix of estimate:
 [[ 2.22624328e-12 -4.76686668e-12]
 [-4.76686668e-12  1.35555083e-11]]


In [23]:
import numpy as np
from pyproj import Geod

geod = Geod(ellps="WGS84")

def theoretical_derivatives(lat1, lon1, lat2, lon2):
    """
    Compute derivatives of geodesic distance with respect to endpoint 2
    All inputs must be scalars or same-sized arrays
    Returns: ds/dlat2, ds/dlon2 in meters per radian
    """
    # Get geodesic parameters - all inputs must have same shape
    fwd_azi, back_azi, distance = geod.inv(lon1, lat1, lon2, lat2)

    # Convert to radians
    lat2_rad = np.radians(lat2)
    # Use back azimuth at point 2 (azi2 in our notation)
    azi2_rad = np.radians(back_azi)

    # WGS84 parameters
    a = geod.a  # semi-major axis
    f = geod.f  # flattening
    e2 = f * (2 - f)  # first eccentricity squared

    # Trigonometric components
    sin_lat2 = np.sin(lat2_rad)
    cos_lat2 = np.cos(lat2_rad)
    sin_azi2 = np.sin(azi2_rad)
    cos_azi2 = np.cos(azi2_rad)

    # Radii of curvature at point 2
    nu2 = 1 - e2 * sin_lat2**2
    N2 = a / np.sqrt(nu2)  # Normal radius (east-west)
    M2 = a * (1 - e2) / (nu2**1.5)  # Meridional radius (north-south)

    # Derivatives in meters per radian
    ds_dlat2 = M2 * cos_azi2  # ρ₂ cos(α₂)
    ds_dlon2 = N2 * cos_lat2 * sin_azi2 # ν₂ cos(φ₂) sin(α₂)

    return ds_dlat2, ds_dlon2

def jacobian_tdoa(receivers_lat, receivers_lon, source_lat, source_lon):
    """
    Compute Jacobian matrix for TDOA system using vectorized operations
    """
    n_receivers = len(receivers_lat)

    # Create arrays with same size for geod.inv
    source_lat_array = np.full(n_receivers, source_lat)
    source_lon_array = np.full(n_receivers, source_lon)

    # Compute all derivatives at once using vectorization
    dsi_dlat, dsi_dlon = theoretical_derivatives(
        source_lat_array, source_lon_array,  # source positions (repeated)
        receivers_lat, receivers_lon         # all receiver positions
    )

    # Reference receiver derivatives (index 0)
    ds0_dlat = dsi_dlat[0]
    ds0_dlon = dsi_dlon[0]

    # Build Jacobian matrix
    J = np.zeros((n_receivers, 2))

    for i in range(n_receivers):
        if i == 0:
            # Reference receiver: s_0 - s_0 = 0
            J[i, 0] = 0.0
            J[i, 1] = 0.0
        else:
            # Difference derivatives: ∂(s_i - s_0)/∂source
            J[i, 0] = dsi_dlat[i] - ds0_dlat  # ∂(s_i - s_0)/∂lat
            J[i, 1] = dsi_dlon[i] - ds0_dlon  # ∂(s_i - s_0)/∂lon

    return J

def compute_distances_vectorized(source_lat, source_lon, receivers_lat, receivers_lon):
    """
    Compute distances from source to all receivers using vectorized geod.inv
    """
    n_receivers = len(receivers_lat)

    # Create arrays with same size
    source_lat_array = np.full(n_receivers, source_lat)
    source_lon_array = np.full(n_receivers, source_lon)

    # Vectorized distance computation
    _, _, distances = geod.inv(
        source_lon_array, source_lat_array,  # source positions (repeated)
        receivers_lon, receivers_lat         # all receiver positions
    )

    return distances

def weighted_least_squares(J, y, W):
    """
    Solve weighted least squares: min ||W^(1/2)(Jx - y)||^2
    """
    N = J.T @ W @ J
    if np.linalg.det(N) < 1e-12:
        raise ValueError("Normal matrix is singular")

    N_inv = np.linalg.inv(N)
    x_hat = N_inv @ J.T @ W @ y

    # Residuals and variance factor
    v = J @ x_hat - y
    n, p = J.shape
    if n > p:
        sigma0_sq = (v.T @ W @ v) / (n - p)
    else:
        sigma0_sq = 0
    # Covariance of parameters
    Sigma_x = sigma0_sq * N_inv

    return x_hat, sigma0_sq, Sigma_x, v

def estimate_position_tdoa(receivers_lat_deg, receivers_lon_deg, tdoa_obs_sec,
                          speed_of_sound, initial_guess_deg, sigma_timing_sec,
                          max_iterations=100, convergence_threshold=1e-10):

    # Convert to numpy arrays
    receivers_lat = np.array(receivers_lat_deg)
    receivers_lon = np.array(receivers_lon_deg)
    tdoa_obs = np.array(tdoa_obs_sec)

    # Handle speed of sound
    if np.isscalar(speed_of_sound):
        speeds = np.full(len(receivers_lat), speed_of_sound)
    else:
        speeds = np.array(speed_of_sound)

    # Convert TDOA to range differences (meters)
    range_diff_obs = np.zeros(len(tdoa_obs))
    for i in range(len(tdoa_obs)):
        range_diff_obs[i] = speeds[i] * tdoa_obs[i]

    # Weight matrix (assuming uncorrelated timing errors)
    sigma_range = speeds * sigma_timing_sec
    W = np.diag(1.0 / sigma_range**2)

    # Initialize
    current_lat, current_lon = initial_guess_deg

    for iteration in range(max_iterations):
        # Compute predicted distances using vectorized function
        predicted_ranges = compute_distances_vectorized(current_lat, current_lon, receivers_lat, receivers_lon)

        # Range differences relative to receiver 0
        predicted_diff = predicted_ranges - predicted_ranges[0]
        # Observation vector (observed - predicted)
        y = range_diff_obs - predicted_diff

        J = jacobian_tdoa(receivers_lat, receivers_lon, current_lat, current_lon)
        # Convert Jacobian from radians to degrees
        J_deg = J.copy()
        J_deg[:, 0] = J[:, 0]* 180/np.pi
        J_deg[:, 1] = J[:, 1] *180/np.pi
        # Solve weighted least squares
        try:
            delta_pos, sigma0_sq, cov_pos, residuals = weighted_least_squares(J_deg, y, W)
        except ValueError as e:
            print(f"Error in iteration {iteration}: {e}")
            break
        # Update position
        current_lat += delta_pos[0]
        current_lon += delta_pos[1]

        # Check convergence
        position_change = np.linalg.norm(delta_pos)

        if position_change < convergence_threshold:
            break
    else:
        print(f"Maximum iterations ({max_iterations}) reached")

    return [current_lat, current_lon], sigma0_sq, cov_pos


# Example usage
if __name__ == "__main__":
    # Receiver positions around Paris (degrees)
    receivers_lat_deg =[-29.0473, -24.2053, -33.5175, -24.3792, -46.6015, -42.9513, -31.5758, -56.4602]
    receivers_lon_deg =[54.258 , 63.0102 ,70.8657 ,72.372,  60.5475, 74.5975 ,83.2423 ,62.976 ]

    # TDOA observations (seconds, relative to receiver 0)
    tdoa_obs_sec = [   0. ,  327.25641012,  509.96581221,  775.34615421, 827.73076916,  992.43589711, 1294.27350402, 1582.29914498]

    # Speed of sound (m/s)
    speed_of_sound = 1480

    # Initial guess
    initial_guess_deg = [-30., 54.]

    # Timing precision
    sigma_timing_sec = 2 # 10 microseconds

    # Test both versions
    estimated_pos, variance_factor, covariance = estimate_position_tdoa(
        receivers_lat_deg, receivers_lon_deg, tdoa_obs_sec,
        speed_of_sound, initial_guess_deg, sigma_timing_sec
    )
    print(f"Estimated position: {estimated_pos[0]:.8f}°N, {estimated_pos[1]:.8f}°E")
    print(f"A posteriori variance factor: {variance_factor:.6f}")
    print(f"Position standard deviations:")
    print(f"  Latitude:  ±{np.sqrt(covariance[0,0]):.2e}° = ±{np.sqrt(covariance[0,0])*111000:.3f}m")
    print(f"  Longitude: ±{np.sqrt(covariance[1,1]):.2e}° = ±{np.sqrt(covariance[1,1])*111000*np.cos(np.radians(estimated_pos[0])):.3f}m")

Maximum iterations (100) reached
Estimated position: -30.07430919°N, 54.18643423°E
A posteriori variance factor: 18.579482
Position standard deviations:
  Latitude:  ±1.30e-05° = ±1.440m
  Longitude: ±2.83e-05° = ±2.715m
